In [2]:
!pip install transformers accelerate evaluate datasets loralib git+https://github.com/huggingface/peft -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import transformers
import accelerate
import peft
print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")

Transformers version: 4.30.1
Accelerate version: 0.20.3
PEFT version: 0.4.0.dev0


In [5]:
model_checkpoint = "google/vit-base-patch16-224-in21k"  # pre-trained model from which to fine-tune
from datasets import load_dataset

# dataset = load_dataset("food101", split="train[:5000]")


In [6]:
dataset = load_dataset("mnist", split="train[:20000]")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to /root/.cache/huggingface/datasets/mnist/mnist/1.0.0/9d494b7f466d6931c64fb39d58bb1249a4d85c9eb9865d9bc20960b999e2a332. Subsequent calls will reuse this data.


In [7]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'2'

In [8]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)
image_processor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [9]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
train_transforms = Compose(
    [
        RandomResizedCrop(image_processor.size["height"]),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

val_transforms = Compose(
    [
        Resize(image_processor.size["height"]),
        CenterCrop(image_processor.size["height"]),
        ToTensor(),
        normalize,
    ]
)


def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch


def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [10]:
# split up training into training + validation
splits = dataset.train_test_split(test_size=0.1)
train_ds = splits["train"]
val_ds = splits["test"]

In [11]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [12]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [13]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
print_trainable_parameters(model)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 85806346 || all params: 85806346 || trainable%: 100.00


In [14]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=1024,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

trainable params: 37764116 || all params: 123562772 || trainable%: 30.56


In [ ]:
for name, param in lora_model.named_parameters():
  print(name)


base_model.model.vit.embeddings.cls_token
base_model.model.vit.embeddings.position_embeddings
base_model.model.vit.embeddings.patch_embeddings.projection.weight
base_model.model.vit.embeddings.patch_embeddings.projection.bias
base_model.model.vit.encoder.layer.0.attention.attention.query.weight
base_model.model.vit.encoder.layer.0.attention.attention.query.bias
base_model.model.vit.encoder.layer.0.attention.attention.query.lora_A.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.query.lora_B.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.key.weight
base_model.model.vit.encoder.layer.0.attention.attention.key.bias
base_model.model.vit.encoder.layer.0.attention.attention.value.weight
base_model.model.vit.encoder.layer.0.attention.attention.value.bias
base_model.model.vit.encoder.layer.0.attention.attention.value.lora_A.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.value.lora_B.default.weight
base_model.model.vit.enc

In [15]:
from transformers import TrainingArguments, Trainer


model_name = model_checkpoint.split("/")[-1]
batch_size = 128

args = TrainingArguments(
    f"{model_name}-finetuned-lora-mninst-r1024",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    num_train_epochs=6,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    label_names=["labels"],
)

In [16]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [17]:
import torch


def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)
train_results = trainer.train()

Cloning https://huggingface.co/riyasankhe/vit-base-patch16-224-in21k-finetuned-lora-mninst-r1024 into local empty directory.


Download file pytorch_model.bin:   0%|          | 17.4k/471M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.93k/3.93k [00:00<?, ?B/s]

Download file runs/Jun11_18-53-08_89758978708c/events.out.tfevents.1686509616.89758978708c.191.0: 100%|#######…

Clean file training_args.bin:  25%|##5       | 1.00k/3.93k [00:00<?, ?B/s]

Clean file runs/Jun11_18-53-08_89758978708c/events.out.tfevents.1686509616.89758978708c.191.0:  15%|#5        …

Clean file pytorch_model.bin:   0%|          | 1.00k/471M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,0.572100,0.098177,0.973500
1,0.416000,0.078475,0.977500
2,0.389500,0.055163,0.982000
4,0.372200,0.045585,0.985500
4,0.341400,0.041499,0.989000


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 10>:10                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1938 in _inner_training_loop     │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2770 in training_step            │
│                                                                                                  │
│   2767 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2768 │   │   │   │   scaled_loss.backward()                                                    │
│   2769 │   │   else:                                                                             │
│ ❱ 2770 │   │   │   self.accelerator.backward(loss)                                               │
│   2771 │   │                                                                                     │
│   2772 │   │   return loss.detach() / self.args.gradient_accumulation_steps                      │
│   2773                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:1819 in backward               │
│                                                                                                  │
│   1816 │   │   elif self.distributed_type == DistributedType.MEGATRON_LM:                        │
│   1817 │   │   │   return                                                                        │
│   1818 │   │   elif self.scaler is not None:                                                     │
│ ❱ 1819 │   │   │   self.scaler.scale(loss).backward(**kwargs)                                    │
│   1820 │   │   else:                                       

In [ ]:
for name, param in lora_model.named_parameters():
  if param.requires_grad:
    print(name)

base_model.model.vit.encoder.layer.0.attention.attention.query.lora_A.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.query.lora_B.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.value.lora_A.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.value.lora_B.default.weight
base_model.model.vit.encoder.layer.1.attention.attention.query.lora_A.default.weight
base_model.model.vit.encoder.layer.1.attention.attention.query.lora_B.default.weight
base_model.model.vit.encoder.layer.1.attention.attention.value.lora_A.default.weight
base_model.model.vit.encoder.layer.1.attention.attention.value.lora_B.default.weight
base_model.model.vit.encoder.layer.2.attention.attention.query.lora_A.default.weight
base_model.model.vit.encoder.layer.2.attention.attention.query.lora_B.default.weight
base_model.model.vit.encoder.layer.2.attention.attention.value.lora_A.default.weight
base_model.model.vit.encoder.layer.2.attention.attention.value.lo

In [ ]:
import numpy as np
from scipy.linalg import svd
u, s, vh = svd(val.cpu())

In [ ]:
u

array([[-8.38902891e-02,  2.52828956e-01, -3.03523988e-01,
         5.29481913e-04, -5.42989016e-01,  5.25742285e-02,
        -7.20960647e-02,  1.11977130e-01, -4.91295308e-01,
         1.01632960e-01, -2.67725706e-01, -2.02883268e-04,
         1.13115460e-01, -4.04948473e-01, -1.29653171e-01,
        -6.78929836e-02],
       [-3.31021309e-01,  3.36001158e-01, -1.72257692e-01,
        -1.33681342e-01,  2.76832342e-01,  6.80933967e-02,
        -1.88365519e-01, -2.44711190e-01, -7.74639547e-02,
         9.06954519e-03,  3.16227615e-01, -1.08388290e-01,
        -3.34605500e-02, -2.21825495e-01,  2.03885287e-02,
         6.20301604e-01],
       [ 1.05929121e-01,  3.66096705e-01,  5.29973507e-02,
        -2.28831381e-01,  1.24555826e-01, -7.72323832e-02,
         1.70040846e-01,  4.29473817e-01,  3.31352986e-02,
        -2.90413797e-01,  8.98646712e-02, -2.88934290e-01,
        -1.02980860e-01, -2.55271018e-01,  5.19856036e-01,
        -2.04263881e-01],
       [ 2.44791180e-01,  2.35114679e

In [19]:
repo_name = f"riyasankhe/{model_name}-finetuned-lora-mninst-r1024"
lora_model.push_to_hub(repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'README.md'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/151M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/151M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/riyasankhe/vit-base-patch16-224-in21k-finetuned-lora-mninst-r1024/commit/9192b364aa86f1e3726afd555325f25b15d92d6c', commit_message='Upload model', commit_description='', oid='9192b364aa86f1e3726afd555325f25b15d92d6c', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
from peft import LoraConfig, get_peft_model

config_2 = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model_2 = get_peft_model(model, config_2)
print_trainable_parameters(lora_model_2)

trainable params: 147456 || all params: 85961492 || trainable%: 0.17


In [22]:
from transformers import TrainingArguments, Trainer


model_name = model_checkpoint.split("/")[-1]
batch_size = 128

args = TrainingArguments(
    f"{model_name}-finetuned-lora-mninst-r4",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    num_train_epochs=7,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    label_names=["labels"],
)

In [23]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)
train_results = trainer.train()

Cloning https://huggingface.co/riyasankhe/vit-base-patch16-224-in21k-finetuned-lora-mninst-r4 into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,1.278100,0.842993,0.917000
1,0.846400,0.547475,0.968500
2,0.755500,0.453896,0.981500
4,0.689100,0.435709,0.975000
4,0.680300,0.401517,0.985000
5,0.651400,0.392259,0.987000
6,0.642300,0.386028,0.987000


In [24]:
repo_name = f"riyasankhe/{model_name}-finetuned-lora-mninst-r4"

# "riyasankhe/vit-base-patch16-224-in21k-finetuned-lora-mninst-r8"
lora_model_2.push_to_hub(repo_name)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/639k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/riyasankhe/vit-base-patch16-224-in21k-finetuned-lora-mninst-r4/commit/38e9fe2f50c9ec1bbb935b408b61ef522563fabe', commit_message='Upload model', commit_description='', oid='38e9fe2f50c9ec1bbb935b408b61ef522563fabe', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
repo_name = "riyasankhe/vit-base-patch16-224-in21k-finetuned-lora-mninst-r8"

from peft import PeftConfig, PeftModel


config = PeftConfig.from_pretrained(repo_name)
model = model = AutoModelForImageClassification.from_pretrained(
    config.base_model_name_or_path,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
# Load the Lora model
inference_model = PeftModel.from_pretrained(model, repo_name)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_errors.py:259 in                  │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   256 │   </Tip>                                                                                 │
│   257 │   """                                                                                    │
│   258 │   try:                                                                                   │
│ ❱ 259 │   │   response.raise_for_status()                                                        │
│   260 │   except HTTPError as e:                                                                 │
│   261 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   262                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/requests/models.py:960 in raise_for_status               │
│                                                                                                  │
│   957 │   │   │   http_error_msg = u'%s Server Error: %s for url: %s' % (self.status_code, rea   │
│   958 │   │                                                                                      │
│   959 │   │   if http_error_msg:                                                                 │
│ ❱ 960 │   │   │   raise HTTPError(http_error_msg, response=self)                                 │
│   961 │                                                                                          │
│   962 │   def close(self):                                                                       │
│   963 │   │   """Releases the connection back to the pool. Once this method has been             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 404 Client Error: Not Found for url: 
https://huggingface.co/riyasankhe/vit-base-patch16-224-in21k-finetuned-lora-mninst-r8/resolve/main/adapter_config.j
son

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/peft/utils/config.py:106 in from_pretrained              │
│                                                                                                  │
│   103 │   │   │   config_file = os.path.join(path, CONFIG_NAME)                                  │
│   104 │   │   else:                                                                              │
│   105 │   │   │   try:                                                                           │
│ ❱ 106 │   │   │   │   config_file = hf_hub_download(                                             │
│   107 │   │   │   │   │   pretrained_model_name_or_path, CONFIG_NAME, subfolder=subfolder, **k   │
│   108 │   │   │   │   )                                                                          │
│   109 │   │   │   except Exception:                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:118 in _inner_fn    │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:                                                           │
│   116 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   117 │   │          